## Improved-RAG

In this project I aim to improved the architecture of RAG. yeah

## Importin da Library

In [1]:
from langchain_openai import ChatOpenAI
from glob import glob
import faiss
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from tqdm import tqdm
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).

  from pandas.core.computation.check import NUMEXPR_INSTALLED

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).

  from pandas.core import (


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

In [3]:
import re

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


In [6]:
os.environ["TOGETHER_API_KEY"] = getpass.getpass()

········


## Document Loader + Splitter + Chunking + Ingesting the Chunks into VectorDB(FAISS)

In [7]:
import PyPDF2
from langchain_together import TogetherEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from glob import glob
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS


In [8]:
paper_paths = glob("/Users/komangandikawirasantosa/Documents/Project/Improved - RAG/PEDF/*.pdf")
pages = []
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-32k-retrieval",
)

text_splitter = SemanticChunker(embeddings)

for path in paper_paths:
    try:
        loader = PyPDFLoader(path)
        doc = loader.load()

        # Extract text content from the document
        text_content = "".join([page.page_content for page in doc])
        
        text_splitter = SemanticChunker(embeddings)
        chunked_documents = text_splitter.create_documents([text_content])
        
        pages.extend(chunked_documents)
    except Exception as e:
        print('Skipping', path, e)


## Ingesting the Chunks into VectorDB + Creating A Retriever

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [10]:
db = FAISS.from_documents(
    pages,
    embeddings
)

In [11]:
retriever = db.as_retriever()

## Using Widely Used Chain without Query Rewriter

In [12]:
llm = ChatOpenAI(
        base_url="https://api.together.xyz/v1",
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0.1,
    )

In [13]:
# Using XML(such as <h1> </h1>) tags as a way of prompt engineering
template = """
<instruction>
Answer the question based on the provided context
</instruction>

Here is the context:
<context>
{context}
</context>

Here is the question:
<question>
{question}
</question>
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# First - General Question
input_query = "What are the main challenges developers face in software development related to bug fixing?"
output = chain.invoke(input_query)

print(output)

According to the provided context, the main challenges developers face in software development related to bug fixing are:



1. Identifying and fixing bugs in code can be a time-consuming and complicated process.

2. Ensuring that the generated fixes are correct and do not introduce new bugs.

3. Improving the performance of automated program repair systems, particularly in terms of their speed and scalability.



These challenges are mentioned in the context as part of the introduction and literature review sections.


In [17]:
# Second - General Question
input_query = "How do deep learning-based program repair tools differ from traditional APR methods?"
output = chain.invoke(input_query)

print(output)

<answer>

Deep learning-based program repair tools differ from traditional APR methods in that they learn bug fixing patterns from existing databases and treat the automated program repair problem as a neural machine translation task, producing a ranking of patches. Unlike traditional approaches, generated patches from DL-based program repair tools are not usually evaluated against a test suite or other automated verification strategy, so they may not even compile. This approach has shown competitive results to traditional approaches, but the quality of the suggestions is still unclear.


In [18]:
# With no Distraction
input_query = "Why Gaussian Assumption is important?"
output = chain.invoke(input_query)

print(output)

According to the provided context, the Gaussian Assumption is important in ED A (Exploratory Data Analysis) because it allows us to:



1. Identify outliers using the three-sigma edit rule.

2. Characterize the distribution of the data using the Gaussian Assumption, which is useful when the data conforms to a Gaussian distribution and helps detect outliers and skewness in the data.



This is stated in the first point of the context: "1. Why is the Gaussian Assumption important in ED A? Numerical data often has imprecision or inaccuracy, which can be handled by using Gaussian Assumption. In ED A, the Gaussian Assumption is important because it allows us to identify outliers using three-sigma edit rule and characterize the distribution of the data using Gaussian probability density function."


In [19]:
# Distraction Question
input_query = "QuixBug is a benchmark for automatic program repair! Why Gaussian Assumption is important?"
output = chain.invoke(input_query)

print(output)

There is no mention of the Gaussian Assumption in the provided context. The context discusses QuixBug as a benchmark for evaluating automatic program repair techniques, but it does not mention the Gaussian Assumption.


As we can see because of the query containing the other chuk or document, it retrieved the wrong chunk resulting in bad response from the LLM

## With input Rewriter

### Using Rewrite-Retrieve-Read Implementation

Thanks to this github from efriss staright from langchain community for this implementation: <br>
https://github.com/langchain-ai/langchain/blob/master/cookbook/rewrite.ipynb

I would like to also say thank you to Roger Oliol for the idea of implementing Few Shot in query rewriting part: <br>
https://dev.to/rogiia/build-an-advanced-rag-app-query-rewriting-h3p

Also big thank you for the research paper from Xinbei MA and her friends talking about query rewriting: <br>
https://arxiv.org/abs/2305.14283

In [23]:
template = """
<instruction>
1. You are a rewriter specialist
2. Rewrite the question for better search query by removing distraction in the question or only extracting the question
3. Follow the output example
4. Only output the rewrited question
</instruction>

here are the output example:
<output_example>
question 1: "How tall is the Eiffel Tower? It looked so high when i was there last year"
answer 1: "What is the height of the Eiffel Tower?"

question 2: "1 oz is 28 grams, how many cm is 1 inch?"
answer 2: "convert 1 inch to cm"

question 3: "What's the main point of the article? What did the author try to convey?"
answer 3: "What is the main key point of the article"

question 4: "The Bruno Mars concert last night was dope as hell, what is the purpose EDA in data science?"
answer 4: "What is the purpose EDA in data science?"
</output_example>

Here is the question:
<question>
{x}
</question>
"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [24]:
chain_rewriter = (
    {"x": RunnablePassthrough()}
    | rewrite_prompt
    | llm
    | StrOutputParser()
)

In [25]:
input_query = "That new laptop is the new thing! Why Gaussian Assumption is important?"
output = chain_rewriter.invoke(input_query)
print(output)

"What is the importance of the Gaussian Assumption?"


This seems to show promising result

## RAG Chain with Rewriter

Thanks for this langchain documentation about mu

In [33]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewrite_prompt | llm | StrOutputParser() | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
# First - General Question
input_query = "What are the main challenges developers face in software development related to bug fixing?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

According to the provided context, the main challenges developers face in software development related to bug fixing are:



1. Ensuring that the generated fixes are correct and do not introduce new bugs.

2. Improving the performance of automated program repair systems, particularly in terms of their speed and scalability.



These challenges are mentioned in the context as the current limitations of automated program repair using large language models, such as OpenAI's Codex and ChatGPT.


In [35]:
# Second - General Question
input_query = "How do deep learning-based program repair tools differ from traditional APR methods?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

According to the provided context, deep learning-based program repair tools differ from traditional APR methods in the following ways:



1. **Evaluation of generated patches**: Traditional APR methods typically rely on test suites to verify program correctness or calls to a constraint solver, which can be time-consuming. In contrast, deep learning-based program repair tools learn bug fixing patterns from existing databases and treat the automated program repair problem as a neural machine translation task, producing a ranking of patches. These generated patches are not usually evaluated against a test suite or other automated verification strategy, so they may not even compile.



2. **Learning bug fixing patterns**: Deep learning-based program repair tools learn bug fixing patterns from existing databases, whereas traditional APR methods often rely on manual analysis or other approaches to identify bug fixes.



3. **Processing and extending source code**: Large-scale language models

In [36]:
# With no Distraction
input_query = "Why Gaussian Assumption is important?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

According to the provided context, the Gaussian Assumption is important in ED A (Error Detection and Analysis) because it allows us to:



1. Identify outliers using the three-sigma edit rule.

2. Characterize the distribution of the data using the Gaussian probability density function.



The Gaussian Assumption is useful when the data conforms to a Gaussian distribution and helps to detect outliers and skewness in the data, which is important in ED A.


In [38]:
# Distraction Question
input_query = "QuixBug is a benchmark for automatic program repair! Why Gaussian Assumption is important?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

The context provided does not mention QuixBug, but it does discuss the importance of the Gaussian Assumption in the context of Extreme Data Analysis (EDA). 



According to the context, the Gaussian Assumption is important in EDA because it allows us to:



1. Identify outliers using the three-sigma edit rule.

2. Characterize the distribution of the data using the Gaussian probability density function.



The Gaussian Assumption is useful when the data conforms to a Gaussian distribution and helps detect outliers and skewness in the data, which is important in EDA.


Yeahh it worksss

## Reranker

I need to ingest the pages to the FAISS vector database again but now using index for the pages <br>
Source: https://python.langchain.com/docs/integrations/retrievers/flashrank-reranker/

In [49]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
import cohere

AttributeError: module 'lib' has no attribute 'X509_V_FLAG_NOTIFY_POLICY'

In [50]:
pip show cohere

Name: cohere

Version: 5.9.2

Summary: 

Home-page: 

Author: 

Author-email: 

License: 

Location: /Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages

Requires: httpx-sse, typing_extensions, pydantic-core, boto3, fastavro, pydantic, types-requests, tokenizers, parameterized, httpx, requests

Required-by: 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cohere

co = cohere.Client(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY,
)
docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]
response = co.rerank(
    model="Salesforce/Llama-Rank-V1",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)


In [42]:
retriever_ranker = db.as_retriever(search_kwargs={"k":20})